#  Veri Bilimi 101 ödevi

| İsim | Soyisim|
| ----------- | ----------- |
| Taha Yunus   | TOPLU       |

ödev 4 parçadan oluşuyor sırayla ele alacağız:
1. Kullanacağınız veriyi indirip, okumak
3. Verinizin içindeki eksik ve kategorik değişkenler ile ilgilenip modele besleyeceğimiz hale getirmek (derste gördüklerimizin üzerine de bir şeyler yapmanız hoşumuza gider demiştik, outlier'ları olan bir veride outlier'lar ile ilgili yaptıklarınızı görmek gibi şeyler olabilir mesela)
3. İlgilendiğiniz probleme göre error metriğine karar vermek (derste gördüğümüz RMSE-RMSLE gibi)

5. Verinizi train-validation-test diye bölmek (burada validation ve test'in gerçek hayatı yansıtması çok önemli)
son olarak:
Olabildiğince fazla model denemek ve metriğimizde en iyi yapanı seçmek

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from pandas.api.types import is_string_dtype, is_numeric_dtype

import math

In [2]:
df = pd.read_csv("data/Train.csv",
                   low_memory = False,parse_dates=["saledate"])
df.head()


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.sort_values(by = "saledate")

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
205615,1646770,9500,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
92803,1404019,24000,1169900,7110,132,99.0,1986,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98346,1415646,35000,1262088,3357,132,99.0,1975,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169297,1596358,19000,1433229,8247,132,99.0,1978,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
274835,1821514,14000,1194089,10150,132,99.0,1980,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393123,6260878,13500,1799594,4102,149,2.0,1000,NaN,NaN,2011-12-30,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN
397987,6288376,9750,1872596,4875,149,2.0,1000,NaN,NaN,2011-12-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
392664,6258093,14500,1877553,3170,149,2.0,1988,NaN,NaN,2011-12-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400458,6315563,12500,1869637,26456,149,2.0,2010,NaN,NaN,2011-12-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [3]:
df["SalePrice"] = np.log(df.SalePrice)

In [4]:
#m= RandomForestRegressor(n_jobs =1)
#m.fit(iris.drop("SalePrice",axis=1),iris.SalePrice)

In [6]:
df["UsageBand"]

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: object

In [7]:


#string dtype 1 olan columnlars pandas category tipine çevirir
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c):
            df[n] = c.astype("category").cat.as_ordered()

In [8]:


#train set e train_cats uygulandıktan sonra ayns category değişimlerim olsun diye validation a train e bu uygulanır
def apply_cats (df, train): 
    for n, c in df.items():
        if train[n].dtype == "category":
            df[n] = pd.Categorical (c, categories = train[n].cat.categories, ordered = True)

In [9]:
train_cats(df)

In [10]:
df["UsageBand"] # donusmedi cunku assignment yapmadik henuz

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: category
Categories (3, object): ['High' < 'Low' < 'Medium']

In [11]:
df["UsageBand"].cat.codes # aslinda ddegisti ama arka planda atamayi yapmadik henuz asagida yapiyoruz

0         1
1         1
2         0
3         0
4         2
         ..
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Length: 401125, dtype: int8

In [12]:
def numericalize(df,col,name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes +1

In [13]:
numericalize(df,df["UsageBand"],"UsageBand")

In [14]:
df["UsageBand"]

0         2
1         2
2         1
3         1
4         3
         ..
401120    0
401121    0
401122    0
401123    0
401124    0
Name: UsageBand, Length: 401125, dtype: int8

In [15]:
df["saledate"]

0        2006-11-16
1        2004-03-26
2        2004-02-26
3        2011-05-19
4        2009-07-23
            ...    
401120   2011-11-02
401121   2011-11-02
401122   2011-11-02
401123   2011-10-25
401124   2011-10-25
Name: saledate, Length: 401125, dtype: datetime64[ns]

In [15]:

def add_datepart (df, dt_name, drop=True, time=False):
    dt_column=df[dt_name]
    
    column_dtype = dt_column.dtype
    
    attr= ['year', 'month', 'week', 'day', 'dayofweek', 'dayofyear',
    'is_month_end', 'is_month_start', 'is_quarter_end', 'is_quarter_start', 'is_year_end', 'is_year_start']
    for a in attr:
        df["Date" + a. capitalize()] = getattr(dt_column.dt, a)
    df["Date"+'Elapsed'] = dt_column.astype (np.int64) // 10 ** 9 # bu satir ne yapiyor?
    
    if drop:
        df.drop(dt_name, axis=1, inplace=True)

In [16]:
add_datepart(df,"saledate")

c:\users\taha yunus\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':


In [17]:
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start,DateElapsed
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,2,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,2,950FII,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,1,226,...,26,3,57,False,False,False,False,False,False,1077753600
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,1,PC120-6E,...,19,3,139,False,False,False,False,False,False,1305763200
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,3,S175,...,23,3,204,False,False,False,False,False,False,1248307200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,9.259131,1840702,21439,149,1.0,2005,NaN,0,35NX2,...,2,2,306,False,False,False,False,False,False,1320192000
401121,6333337,9.305651,1830472,21439,149,1.0,2005,NaN,0,35NX2,...,2,2,306,False,False,False,False,False,False,1320192000
401122,6333338,9.350102,1887659,21439,149,1.0,2005,NaN,0,35NX2,...,2,2,306,False,False,False,False,False,False,1320192000
401123,6333341,9.104980,1903570,21435,149,2.0,2005,NaN,0,30NX,...,25,1,298,False,False,False,False,False,False,1319500800


In [18]:
def fix_missing (df, col, name, nan_dict, is_train):
    if is_train:
        if is_numeric_dtype(col): 
            if pd.isnull(col).sum():
                df[name+" na"] = pd.isnull(col) 
                nan_dict[name] = col.median() 
                df [name] = col.fillna (nan_dict[name])
    else:
        if is_numeric_dtype(col):
                if name in nan_dict:
                    df [name+"na"] = pd.isnull(col)
                    df [name] = col.fillna(nan_dict[name])
                else:
                    df[name] = col.fillna (df[name].median())

In [19]:
def proc_df(df, y_fld, nan_dict=None, is_train=True):
    df = df.copy()
    y = df[y_fld].values
    df.drop(y_fld, axis= 1, inplace=True)## tahmin etmeye calistigimizi cikariyoruz!
   
    if nan_dict is None: 
        nan_dict = {}
    
    for n, c in df.items():
        fix_missing (df, c, n,nan_dict, is_train)
        numericalize(df, c, n)
    if is_train:
        return df, y, nan_dict
    return df,    y

In [20]:
def split_train_model(df,n):
    return df[:n].copy(), df[n:].copy()

In [21]:
n_valid = 12000 # validation icin 12k deger
n_train = len(df) - n_valid
raw_valid , raw_train = split_train_model(df,n_train)


In [22]:
x_train,y_train ,nas = proc_df(raw_train,'SalePrice')

In [23]:
x_valid,y_valid  = proc_df(raw_valid,'SalePrice',nan_dict= nas,is_train = False)

In [28]:
m= RandomForestRegressor(n_jobs =-1,n_estimators = 10,min_samples_leaf=5,max_features= 0.5)
m.fit(x_train,y_train)
m.score(x_train,y_train)

0.9435593988192416

In [29]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())


In [30]:
def print_score(m):
    res = [rmse(m.predict(x_train), y_train),
           rmse(m.predict(x_valid), y_valid),
           m.score(x_train, y_train), m.score(x_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
%time m.fit(x_train, y_train)
print_score(m)


Wall time: 305 ms
[0.17782399038601862, 0.4689021456872003, 0.9435285840533009, 0.5404854512216286]
